**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Calcule los valores máximo y mínimo de la tercera columna por letra.

In [ ]:
%%writefile input.txt
B   1999-08-28   14.1
E   1999-12-06   12.2
E   1993-07-21   17.0
C   1991-02-12   13.8
E   1995-04-25   16.9
A   1992-08-22   14.8
B   1999-06-11   12.1
E   1993-01-27   13.2
E   1999-09-10   11.3
E   1990-05-03   16.4
E   1994-02-14   10.5
A   1988-04-27   12.6
A   1990-10-06   10.7
E   1985-02-12   16.8
E   1998-09-14   16.9
B   1994-08-30   17.0
A   1997-12-15   13.6
B   1995-08-23   10.7
B   1998-11-22   13.8
B   1997-04-09   14.9
E   1993-12-27   18.4
E   1999-01-14   15.3
A   1992-09-19   18.2
B   1993-03-02   14.4
B   1999-10-21   13.5
A   1990-08-31   12.6
C   1994-01-25   10.7
E   1990-02-09   18.8
A   1990-09-26   14.9
A   1993-05-08   16.8
B   1995-09-06   14.7
E   1991-02-18   14.6
A   1993-01-11   14.5
A   1990-07-22   18.4
C   1994-09-09   15.3
C   1994-07-27   10.2
D   1990-10-10   15.1
A   1990-09-05   11.0
B   1991-10-01   15.0
A   1994-10-25   13.1

## Mapper

In [13]:
%%writefile mapper.py
#! /usr/bin/env python3
import sys
import itertools

class Mapper():
    def __init__(self,stream):
        self.stream=stream
    def emit(self,key,value):
        sys.stdout.write("{},{}\n".format(key,value))
    def __iter__(self):
        for line in self.stream:
            key=line.split(" ")[0]
            val=float(line.split(" ")[6])
            yield(key,val)
    def map(self):
        for key, val in self:
            self.emit(key=key,value=val)

if __name__ == "__main__": 
    mapper=Mapper(sys.stdin)
    mapper.map()

Overwriting mapper.py


In [14]:
!chmod +x mapper.py

In [15]:
!cat ./input.txt | ./mapper.py | sort

A,10.0
A,11.0
A,12.0
A,12.0
A,13.0
A,13.0
A,14.0
A,14.0
A,14.0
A,16.0
A,18.0
A,18.0
B,10.0
B,12.0
B,13.0
B,13.0
B,14.0
B,14.0
B,14.0
B,14.0
B,15.0
B,17.0
C,10.0
C,10.0
C,13.0
C,15.0
D,15.0
E,10.0
E,11.0
E,12.0
E,13.0
E,14.0
E,15.0
E,16.0
E,16.0
E,16.0
E,16.0
E,17.0
E,18.0
E,18.0


## Reducer

In [22]:
%%writefile reducer.py
#!/usr/bin/env python3

import sys
import itertools

class Reducer():
    def __init__(self,stream):
        self.stream=stream
    def emit(self,key,maximo,minimo):
        sys.stdout.write("{},{},{}\n".format(key,maximo,minimo))
    def __iter__(self):
        for line in self.stream:
            key=line.split(",")[0]
            value=float(line.split(",")[1])
            yield(key,value)
    def reduce(self):
        for key, group in itertools.groupby(self,lambda x:x[0]):
            total=[]
            for key, value in group:
                total.append(value)
            self.emit(key=key, maximo=max(total),minimo=min(total))
                

if __name__ == '__main__': 
    reducer=Reducer(sys.stdin)
    reducer.reduce()

Overwriting reducer.py


In [23]:
!chmod +x reducer.py

In [24]:
!cat ./input.txt | ./mapper.py | sort | ./reducer.py

A,18.0,10.0
B,17.0,10.0
C,15.0,10.0
D,15.0,15.0
E,18.0,10.0


## Ejecución

In [ ]:
%%bash
rm -rf output
STREAM=$HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar
chmod +x mapper.py
chmod +x reducer.py
hadoop jar $STREAM -input input.txt -output output  -mapper mapper.py -reducer reducer.py
cat output/part-00000